# Initialize AlexNet Model

In [1]:
import sys, os
sys.path.append('..')

from utils.models.alexnet import alexnet
from keras.utils import plot_model
from keras.optimizers import SGD

model = alexnet()
model.summary()
plot_model(model, to_file="alexnet_model.png", show_shapes=True)

# copile model
opt = SGD(lr=0.05)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])



Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 384)        

# Load dataset

In [2]:
import numpy as np
from imutils import paths



from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader


dataset_path = '../datasets/plantVillage-tomato-mohanty'

# Get the list of image names
image_paths = list(paths.list_images(dataset_path))

print(len(image_paths), 'images loaded')

# Get unique classnames
classNames = [pt.split(os.path.sep)[-2] for pt in image_paths]
classNames = [str(x) for x in np.unique(classNames)]

# Preprocessors
aap = AspectAwarePreprocessor(227, 227)
itap = ImageToArrayPreprocessor()

# Add preprocessors to DataSetLoader class
sdl = SimpleDatasetLoader(preprocessors=[aap, itap])

(data, labels) = sdl.load(image_paths, verbose=500)
data = data.astype('float') / 255.0

print(classNames)


18160 images loaded
[INFO]: Processed 500/18160
[INFO]: Processed 1000/18160
[INFO]: Processed 1500/18160
[INFO]: Processed 2000/18160
[INFO]: Processed 2500/18160
[INFO]: Processed 3000/18160
[INFO]: Processed 3500/18160
[INFO]: Processed 4000/18160
[INFO]: Processed 4500/18160
[INFO]: Processed 5000/18160
[INFO]: Processed 5500/18160
[INFO]: Processed 6000/18160
[INFO]: Processed 6500/18160
[INFO]: Processed 7000/18160
[INFO]: Processed 7500/18160
[INFO]: Processed 8000/18160
[INFO]: Processed 8500/18160
[INFO]: Processed 9000/18160
[INFO]: Processed 9500/18160
[INFO]: Processed 10000/18160
[INFO]: Processed 10500/18160
[INFO]: Processed 11000/18160
[INFO]: Processed 11500/18160
[INFO]: Processed 12000/18160
[INFO]: Processed 12500/18160
[INFO]: Processed 13000/18160
[INFO]: Processed 13500/18160
[INFO]: Processed 14000/18160
[INFO]: Processed 14500/18160
[INFO]: Processed 15000/18160
[INFO]: Processed 15500/18160
[INFO]: Processed 16000/18160
[INFO]: Processed 16500/18160
[INFO]: Pr

# Encode labels and split dataset

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# Split data into training (75%) and testing (25%) data
(train_x, test_x, train_y, test_y) = train_test_split(data, labels,
test_size=0.25, random_state=42)

# Convert the labels from integers to vectors
lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y = lb.transform(test_y)


# Fit model

In [4]:
# train the network
print("[INFO] training network...")
H = model.fit(train_x, train_y, validation_data=(test_x, test_y),
              batch_size=32, epochs=100, verbose=1)

model.save('alexnet_model.h5')

[INFO] training network...


ValueError: Error when checking target: expected activation_9 to have shape (17,) but got array with shape (10,)

# Generate classfication report

In [ ]:
from sklearn.metrics import classification_report


# If one hot encoding is used
print(classification_report(test_y, model.predict(test_x),
                            target_names=classNames))


# Plot model

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 100), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["output"])
plt.show()